In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import regex as re
import time

In [2]:
def counter(df, label):
  word_count = {}
  for cat in categories:
    word_dict = {}
    data = df[df[label] == cat]['headline'] +' '+ df[df[label] == cat]['short_description']
    for row in data:
      words = list(set(re.split("[^a-zA-Z]",row)))#.split(' ')
      for word in words:
        word = word.lower()
        if word in word_dict:
          word_dict[word] = word_dict[word]+1
        else:
          word_dict[word] = 1
    for word, count in dict(word_dict).items():
      if (len(word)<3) or (count<4):
        word_dict.pop(word)
    word_count[cat] = word_dict
  return pd.DataFrame(word_count).fillna(0)

In [3]:
def prob_cond(word, category, word_df, a=0):
  """word: word whose probability needs to be found
     category: prior condition of the probabilty
     word_df: dictionary of the count of word in categories
     a: laplace smoothing factor"""
  word = word.lower()
  category = category.upper()
  if word in vocab:
    prob = (word_df.loc[word,category]+a)/(word_df[category].sum()+a*len(word_df[category]))
  else:
    prob = (a)/(word_df[category].sum()+a*len(word_df[category]))
  return prob

In [4]:
def prob(word, prob_df):
  word = word.lower()
  probblty = 0
  try:
    for cat in prob_df.columns:
      probblty += prob_df.loc[word,cat]*p_cat[cat]
    return probblty
  except:
    return probblty


In [5]:
def train(df,label='category', smoothing = 1):
  global categories
  categories = df[label].unique()
  temp = df[label].value_counts(sort=False)/df[label].value_counts().sum()
  p_cat = temp.to_dict()

  word_count = counter(df, label)
  global vocab
  vocab = word_count.index
  pc_word = {}
  for cat in categories:
    word_prob = {}
    for word in vocab:#word_count[cat].keys():
      word_prob[word] = prob_cond(word,cat,word_count,a=smoothing)
    pc_word[cat] = word_prob
  pc_word = pd.DataFrame(pc_word)
  return p_cat, word_count, pc_word

In [6]:
def predict(X):
  cat_pred = np.array([])
  #data = X#df_val['headline']#df_val['short_description']+' '+df_val['headline']
  for row in X:
    val_words = list(set(re.split("[^a-zA-Z]",row)))
    for word in val_words:
      if (len(word)<3):
        val_words.remove(word)
    cat_prob = np.array([])
    for cat in categories:
      list_prob = np.array([])
      for word in val_words:
        if word in vocab:#try:
          list_prob = np.append(list_prob,pc_word.loc[word,cat])#,word_count,a=1))
        else:
          list_prob = np.append(list_prob,prob_cond(word,cat,word_count,a=smoothing))
      cat_prob = np.append(cat_prob,np.prod(list_prob)*p_cat[cat])
    cat_prob = cat_prob/sum(cat_prob)
    # id = np.argmax(cat_prob)
    cat_pred = np.append(cat_pred,np.argmax(cat_prob))
  return cat_pred

In [7]:
def accuracy(prediction, actual):
  accu = 0
  for i in range(len(prediction)):
    if prediction[i] == actual[i]:
      accu += 1
  accu = accu/len(prediction)
  return accu

In [8]:
# !wget -O news.json 'https://drive.google.com/uc?export=download&id=1Fjy7lj4WkGDjcHRiuw1yHEPuNekwqWhp'
# # https://drive.google.com/file/d/1Fjy7lj4WkGDjcHRiuw1yHEPuNekwqWhp/view?usp=sharing

In [9]:
df = pd.read_json('../input/news-category-dataset/News_Category_Dataset_v2.json',lines=True)
df.head()

,category,headline,authors,link,short_description,date
0,CRIME,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,ENTERTAINMENT,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,ENTERTAINMENT,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,ENTERTAINMENT,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,ENTERTAINMENT,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [10]:
len(df.category.unique())

41

In [11]:
df.category.unique()

array(['CRIME', 'ENTERTAINMENT', 'WORLD NEWS', 'IMPACT', 'POLITICS',
       'WEIRD NEWS', 'BLACK VOICES', 'WOMEN', 'COMEDY', 'QUEER VOICES',
       'SPORTS', 'BUSINESS', 'TRAVEL', 'MEDIA', 'TECH', 'RELIGION',
       'SCIENCE', 'LATINO VOICES', 'EDUCATION', 'COLLEGE', 'PARENTS',
       'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE', 'HEALTHY LIVING',
       'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST', 'FIFTY', 'ARTS',
       'WELLNESS', 'PARENTING', 'HOME & LIVING', 'STYLE & BEAUTY',
       'DIVORCE', 'WEDDINGS', 'FOOD & DRINK', 'MONEY', 'ENVIRONMENT',
       'CULTURE & ARTS'], dtype=object)

In [12]:
df['category'] = df['category'].replace(['CULTURE & ARTS','THE WORLDPOST','PARENTS','ARTS','STYLE','TASTE','WELLNESS','WORLDPOST'],
                                        ['ARTS & CULTURE','WORLD NEWS','PARENTING','ARTS & CULTURE','STYLE & BEAUTY','FOOD & DRINK','HEALTHY LIVING','WORLD NEWS'])

In [13]:
len(df.category.unique())

33

In [14]:
df.category.unique()

array(['CRIME', 'ENTERTAINMENT', 'WORLD NEWS', 'IMPACT', 'POLITICS',
       'WEIRD NEWS', 'BLACK VOICES', 'WOMEN', 'COMEDY', 'QUEER VOICES',
       'SPORTS', 'BUSINESS', 'TRAVEL', 'MEDIA', 'TECH', 'RELIGION',
       'SCIENCE', 'LATINO VOICES', 'EDUCATION', 'COLLEGE', 'PARENTING',
       'ARTS & CULTURE', 'STYLE & BEAUTY', 'GREEN', 'FOOD & DRINK',
       'HEALTHY LIVING', 'GOOD NEWS', 'FIFTY', 'HOME & LIVING', 'DIVORCE',
       'WEDDINGS', 'MONEY', 'ENVIRONMENT'], dtype=object)

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200853 entries, 0 to 200852
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   category           200853 non-null  object        
 1   headline           200853 non-null  object        
 2   authors            200853 non-null  object        
 3   link               200853 non-null  object        
 4   short_description  200853 non-null  object        
 5   date               200853 non-null  datetime64[ns]
dtypes: datetime64[ns](1), object(5)
memory usage: 9.2+ MB


In [16]:
df, df_test = train_test_split(df, test_size=0.1, stratify=df['category'])
# df_val, df_test = train_test_split(df_val, test_size=0.5, random_state=42)

In [17]:
smoothing = 1

In [18]:
# p_cat, word_count, pc_word = train(df, 'category', stpwrds=stpwrds, smoothing=smoothing)
# cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}


In [19]:
# start = time.time()
# input = df_val['headline']#+' '+df_val['short_description']
# predictions = predict(input)
# predictions = [cat_dict[v] for v in predictions]
# print(time.time()-start)

In [20]:
cv= 5
score = []
for i in range(cv):
    df_train, df_val = train_test_split(df, test_size=0.1)#, random_state=42)
    p_cat, word_count, pc_word = train(df_train, 'category', smoothing=smoothing)
    cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}
    input = df_val['headline']#+' '+df_val['short_description']
    predictions = predict(input)
    predictions = [cat_dict[v] for v in predictions]
    actual = df_val['category'].values
    score.append(accuracy(predictions,actual))
np.mean(score)

0.5948221496929801

In [21]:
score

[0.6023123305858273,
 0.5883166454610832,
 0.5958953366155889,
 0.5971676716269293,
 0.5904187641754716]

In [22]:
p_cat, word_count, pc_word = train(df, 'category',smoothing=smoothing)
cat_dict = {k:v for k,v in zip(range(len(categories)),categories)}
print("Probability of 'The' occuring in a headline is: ", round(prob('The',pc_word),3))
print("Probability of 'The' occuring in a 'Business' headline is: ", round(prob_cond('The'.lower(),'BUSINESS'.upper(),word_count, a=smoothing),3))

Probability of 'The' occuring in a headline is:  0.037
Probability of 'The' occuring in a 'Business' headline is:  0.038


In [23]:
input = df_test['headline']+' '+df_test['short_description']
predictions = predict(input)
predictions = [cat_dict[v] for v in predictions]
actual = df_test['category'].values
accuracy(predictions,actual)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


0.6027581399980085

In [24]:
full_prob = []
for cat in categories:
  word_prob = []
  for word in vocab:
    word_prob.append(pc_word.loc[word,cat]*p_cat[cat]/prob(word,pc_word))
  full_prob.append(word_prob)
full_prob = np.array(full_prob)
full_prob_df = pd.DataFrame(full_prob.T,columns=categories,index=vocab)

top_word = []
for cat in categories:
  top_word.append(full_prob_df[cat].sort_values(ascending=False).index)
top_word = np.array(top_word)
top_words_df = pd.DataFrame(top_word.T,columns=categories)

In [25]:
top_words_df.iloc[:10,:16]

,ENTERTAINMENT,PARENTING,RELIGION,POLITICS,FOOD & DRINK,HEALTHY LIVING,ARTS & CULTURE,QUEER VOICES,GREEN,WEDDINGS,DIVORCE,BUSINESS,SPORTS,LATINO VOICES,COMEDY,BLACK VOICES
0,trailer,breastfeeding,francis,democrats,recipes,gps,imageblog,queer,wildfire,brides,divorce,krugman,touchdown,latinos,maher,trayvon
1,thrones,parenting,bishops,huffpollster,recipe,workout,exhibition,lgbtq,elephants,weddings,divorced,jpmorgan,nba,latina,meyers,panther
2,teaser,mommy,interfaith,republicans,cocktails,fitness,paintings,bisexual,carbon,wedding,divorcing,investors,ncaa,latino,colbert,blacks
3,swift,darndest,pope,senate,sauce,workouts,nighter,lesbian,meatless,bride,blended,marketing,mlb,latinas,trevor,mlk
4,avengers,parent,prayers,democrat,dessert,diabetes,theatre,gay,seaworld,bridal,infidelity,managers,championship,hispanic,spoof,luther
5,album,motherhood,bishop,rubio,baked,strain,ballet,lgbt,conservation,groom,cheating,startup,nfl,ferrera,conan,philando
6,khloe,toddlers,catholics,sen,salad,mindfulness,canvas,trans,oceans,nuptials,alimony,startups,tournament,afro,seth,black
7,reboot,toddler,catholic,marco,flavor,fearless,opera,rupaul,fracking,grooms,divorces,tesla,quarterback,shakira,kimmel,ish
8,kylie,moms,worship,mcconnell,tastemakers,chronic,painter,transgender,wildfires,planner,breakup,entrepreneurial,lebron,despacito,noah,duvernay
9,reynolds,babies,christianity,congressman,desserts,nutrition,portraits,drag,tornadoes,knot,exes,marketers,playoff,rodriguez,fallon,castile


In [26]:
top_words_df.iloc[:10,16:]

,TRAVEL,MEDIA,IMPACT,ENVIRONMENT,EDUCATION,WORLD NEWS,STYLE & BEAUTY,WOMEN,FIFTY,HOME & LIVING,GOOD NEWS,CRIME,MONEY,COLLEGE,WEIRD NEWS,SCIENCE,TECH
0,destinations,gawker,homelessness,networx,charter,aleppo,huffpoststyle,succinct,boomer,decor,pup,fatally,prepaid,frat,fark,nasa,samsung
1,hotels,journalism,homeless,tornado,teachers,migrants,stylelist,feminism,boomers,homesubmissions,pit,suspect,credit,fraternity,ufo,spacecraft,iphone
2,traveler,msnbc,aspen,sands,classroom,rohingya,fashion,brighten,midlife,furniture,kittens,gunman,lottery,faculty,hydraulic,astronomers,microsoft
3,travelers,newsroom,charities,tar,education,mosul,tumblr,feminist,aging,pitches,bull,shooter,savings,colleges,snake,rover,android
4,attractions,advertisers,nonprofits,panda,educational,referendum,middleton,steinem,grandkids,crafts,pups,stabbing,finances,universities,alligator,telescope,unconfirmed
5,resorts,journalists,charitable,spill,educators,rebels,wardrobe,metoo,retirement,repurposing,selfless,tsarnaev,mortgage,campuses,squirrel,planets,apple
6,beaches,jazeera,volunteering,animal,repayment,ceasefire,outfit,funniest,grandchildren,sprout,prosthetic,allegedly,loan,campus,pooch,asteroid,google
7,airline,reilly,maternal,keystone,tenure,myanmar,makeup,gloria,menopause,diy,amputee,inmate,expenses,fraternities,burglary,mars,iphones
8,tsa,hannity,mutilation,sloth,curriculum,militants,lipstick,sexism,retirees,decorating,paralyzed,stabbed,refund,professors,weirdest,meteor,software
9,getaways,ailes,philanthropy,species,devos,minister,runway,feminists,retire,appraises,doodle,murdering,jackpot,grads,mph,astronauts,ios
